# Parsing with nom

Resources:

https://iximiuz.com/en/posts/rust-writing-parsers-with-nom/

https://blog.logrocket.com/parsing-in-rust-with-nom/

To return error from parser:
```rust
Err(nom::Err::Error(nom::error::Error::from_error_kind(input, nom::error::ErrorKind::Char)))
```

## 1. URL Parser

In [2]:
extern crate nom;

In [3]:

use nom::bytes::complete::tag;
use nom::character::complete::*;
use nom::multi::*;
use nom::sequence::*;
use nom::IResult;
use nom::branch::alt;
use nom::complete::*;
use nom::InputTakeAtPosition;
use nom::AsChar;


In [103]:
type Res<'a> = IResult<&'a str, &'a str>;
type ResStr<'a> = IResult<&'a str, String>;//, NomError<&'a str>>;

In [51]:
let t : Res = terminated(alphanumeric1, tag("."))("ciao.");


In [52]:
t

Ok(("", "ciao"))

In [53]:
let t : ResStr = 
    many1(terminated(alphanumeric1, tag(".")))("first.second.")
        .map(|(remaining, parsed)| {
            ("", format!("{:?}", parsed))
});

In [54]:
t

Ok(("", "[\"first\", \"second\"]"))

In [56]:
fn is_alpha_char(c: char) -> bool { 
    c.is_alphabetic()
}

In [63]:
let t : ResStr = 
    alt(
        (many1
            (terminated
                (alphanumeric1, tag("."))),
         many_m_n(1, 1, take_while(is_alpha_char))))
    ("first.second.third")
    .map(|(remaining, parsed)| {
            ("", format!("{:?}", parsed))
    });

### Adding hyphen to alphanumeric1
Original code:
```
nom/character/complete.rs
```
```rust
pub fn alphanumeric1<T, E: ParseError<T>>(input: T) -> IResult<T, T, E>
where
  T: InputTakeAtPosition,
  <T as InputTakeAtPosition>::Item: AsChar,
{
  input.split_at_position1_complete(|item| !item.is_alphanum(), ErrorKind::AlphaNumeric)
}  
```

In [73]:

fn alphahyphen1<T>(i: T) -> IResult<T, T>
where
    T: InputTakeAtPosition,
    <T as InputTakeAtPosition>::Item: AsChar,
{
    i.split_at_position1_complete(
        |item| {
            let char_item = item.as_char();
            !(char_item == '-') && !char_item.is_alphanum()
        },
        ErrorKind::AlphaNumeric,
    )
}

In [95]:
let t : ResStr = 
    separated_list1(tag("."), alphahyphen1)("^x.b.c/ccc")
    .map(|(rem, v)| (rem, format!("{:?}", v)));

In [96]:
t

Err(Error(Error { input: "^x.b.c/ccc", code: AlphaNumeric }))

In [118]:
// The map function cannot return an error, need to map to a type like
// enum IP {
//   Address([u8;4]),
//   Invalid
// }
// return empty string in this case
let x : ResStr = 
    separated_list1(tag("."), digit1)("234.12.56.13")
    .map(|(remaining, v)|   {
        if v.iter().any(
            |x| x.parse::<i32>().unwrap() < 1 || 
                x.parse::<i32>().unwrap() > 254) {
            return (remaining, "".to_owned());
            // also check for reserved addresses 10., broadcast...
        }
        else {
            return (remaining, format!("{:?}", v));
        }
    });


In [117]:
x

Ok(("", "[\"234\", \"12\", \"56\", \"13\"]"))

### To return error from parser:
```rust
Err(nom::Err::Error(nom::error::Error::from_error_kind(input, nom::error::ErrorKind::Char)))
```

## 2. Custom Parsers

In [128]:
use nom::error::ParseError;
/// Parses strings like [a-zA-Z_][a-zA-Z0-9_]*
fn ipaddress(input: &str) -> IResult<&str, String> {
    let s = separated_list1(tag("."), digit1)(input);
    if let Err(e) = s {
        return Err(e);
    } else {
        let (rem, p) = s.map(|(remaining, v)|   {
                    if v.iter().any(
                        |x| x.parse::<i32>().unwrap() < 1 || 
                            x.parse::<i32>().unwrap() > 254) {
                        return (remaining, "".to_owned());
                        // also check for reserved addresses 10., broadcast...
                    }
                    else {
                        return (remaining, format!("{:?}", v));
                    }}).unwrap();
        if p.is_empty() {
            Err(nom::Err::Error(nom::error::Error::from_error_kind(rem, nom::error::ErrorKind::TooLarge)))
            // WANT TO USE:
//             Err(nom::Err::Failure(ParseError::partial(
//              "regex",
//              "closing '/' symbol",
//              rest,
//            )))
            // complains about missing 'dyn'
         
        } else {
            Ok((rem, p))
        }
    }
      
}

In [131]:
let x : ResStr = ipaddress("255.6.7.7");

In [132]:
x

Err(Error(Error { input: "", code: TooLarge }))

In [17]:
let ip_address_str = ["194", "221", "A", "12"];
let ip_address = ip_address_str.map(|d| d.parse::<i32>()).into_iter().collect::<Result<Vec<_>,_>>();
match ip_address {
    Ok(_) => print!("OK"),
    _ => print!("No way")
}

Error: consider importing one of these items

In [18]:
let x = [1,4,5,6];